
### In this notebook I will try to repair hyperex amplicon extraction tool

hyperex keeps crushing when there is forward primer found upstream revese primer:

thread 'main' panicked at /home/mg/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/hyperex-0.1.1/src/utils.rs:420:45: slice index starts at 256 but ends at 65 note: run with \`RUST_BACKTRACE=1` environment variable to display a backtrace Aborted (core dumped)   

I will  load the fasta seq with Biopython and then use hyperex on each read

In [4]:
import pandas as pd
import os
from Bio import SeqIO
print(os.getcwd())
from pprint import pprint

/home/mg/ChimeraProject/scripts


#### I can use uchime or unite datasets for Simera, but I will use Unite for reference kmers, so SILVA would be better

#### UPDATE: SILVA aslo sucks because there are no primers in the sequences - it it only SSU and LSU database - no ITS I guess

In [23]:
# only fungi
pathh = "../datasets/silva_data/SILVA_138.2_SSURef_NR99_tax_silva.fasta"
fasta_seq = list(SeqIO.parse(pathh, "fasta"))
print(f"{len(fasta_seq):,}".replace(",", "."))
only_fungi = [i for i in fasta_seq if "fungi" in i.description.lower()]
print(len(only_fungi))
fasta_seq = only_fungi

10302


#### lets look at the unite and uchime sets and if the uchime is only subset of unite

In [2]:
pathh_uchime_untrim = "../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/untrimmed_sequences/uchime_reference_dataset_untrimmed_16_10_2022.fasta"
pathh_unite_its2 = "../datasets/unite_latest/itsx_out/UNITE_public_21.04.2024_ITSx.ITS2.fasta"
pathh_unite_whole = "../datasets/unite_latest/UNITE_public_21.04.2024.fasta"

unite, uchime = list(SeqIO.parse(pathh_unite_whole, "fasta")), list(SeqIO.parse(pathh_uchime_untrim, "fasta"))
#fasta_seq = list(SeqIO.parse(pathh, "fasta"))
#fasta_seq = fasta_seq[:300]
print(f"{len(unite):,}".replace(",", "."),f"{len(uchime):,}".replace(",", "."))


2.030.075 161.335


In [3]:
from pprint import pprint
unite_ids = [i.id.split("|")[0] for i in unite]
uchime_ids = [i.id.split("|")[1] for i in uchime]
print(len(unite_ids), len(uchime_ids))
pprint(unite_ids[:5])
pprint(uchime_ids[:5])
unite_ids_set = set(unite_ids)
uchime_ids_set = set(uchime_ids)

2030075 161335
['UDB016649', 'UDB016650', 'UDB016651', 'UDB016653', 'UDB016654']
['UDB03390183', 'UDB02869914', 'UDB02885146', 'UDB02757411', 'UDB02750185']


In [4]:
intersection = uchime_ids_set & unite_ids_set
print(len(intersection))
diff = uchime_ids_set - unite_ids_set
print(len(diff))

60609
100726


#### there is one milion read that are unique to uchime_untrimmed dataset when compared to UNITE so I will use that

In [5]:
fasta_seq = [i for i in uchime if i.id.split("|")[1] in diff]

### 

In [3]:
import pandas as pd
import re
primers_df = pd.read_excel("../my_data/primers.xlsx",header=1)
primers_df["better_names"] = primers_df["homogenized primer names"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\-\.\_]', '_', str(x).replace("/","___")))
primers_df_filtered = primers_df[~primers_df["primers sequences"].str.contains("[\\(a-z\\+]")]
primers_df_filtered


,homogenized primer names,primers sequences,target marker,Notes,better_names
0,18S-F/5.8S-R,GTAAAAGTCGTAACAAGGTTTC/GTTCAAAGAYTCGATGATTCAC,ITS1,NaN,18S-F___5.8S-R
1,5.8S_Fun/ITS4_Fun,AACTTTYRRCAAYGGATCWCT/AGCCTCCGCTTATTGATATGCTTAART,ITS2,NaN,5.8S_Fun___ITS4_Fun
2,58A2F/ITS4,ATCGATGAAGAACGCAG/TCCTCCGCTTATTGATATGC,ITS2,NaN,58A2F___ITS4
3,fITS7/ITS4,GTGARTCATCGAATCTTTG/TCCTCCGCTTATTGATATGC,ITS2,NaN,fITS7___ITS4
4,fITS9/ITS4,GAACGCAGCRAAIIGYGA/TCCTCCGCTTATTGATATGC,ITS2,fITS9=ITSf9=ITS9F,fITS9___ITS4
5,FSeq/RSeq,ATGCCTGTTTGAGCGTC/CCTACCTGATTTGAGGTC,ITS2,NaN,FSeq___RSeq
6,gITS7/ITS4,GTGARTCATCGARTCTTTG/TCCTCCGCTTATTGATATGC,ITS2,NaN,gITS7___ITS4
10,gITS7/ITS4ngs,GTGARTCATCGARTCTTTG/TCCTSCGCTTATTGATATGC,ITS2,NaN,gITS7___ITS4ngs
11,gITS7ngs/ITS4ngsUni,GTGARTCATCRARTYTTTG/CCTSCSCTTANTDATATGC,ITS2,NaN,gITS7ngs___ITS4ngsUni
12,ITS1/ITS2,TCCGTAGGTGAACCTGCGG/GCTGCGTTCTTCATCGATGC,ITS1,NaN,ITS1___ITS2


In [5]:
primers_df_filtered.better_names.to_clipboard()

In [7]:
from hyperex_repaired import hyperex_repaired

## TEST hyperex_repaired function

In [8]:
#test hyperex_repaired on test_hyperex_repaired.fa
test_fasta_seq = list(SeqIO.parse("test_hyperex_repaired.fa", "fasta"))
test_res_seq = hyperex_repaired(test_fasta_seq,"TCCGTAGGTGAACCTGCGG","GCTGCGTTCTTCATCGATGC")
print("All  reads:\n",", ".join([i.id for i in test_fasta_seq]),sep="")
print("Good reads:\n",", ".join([i.id for i in test_res_seq]),sep="")
print("Bad reads:\n",", ".join(set([i.id for i in test_fasta_seq])-set([i.id for i in test_res_seq])),sep="")
test_res_seq

WARNINGS:
25-times  Sequence length is less than 1500 bp. We may not be able to find some regions
1-times  Region  not found because primer TCCGTAGGTGAACCTGCGG was not found in the sequence
1-times  Region  not found because primer GCTGCGTTCTTCATCGATGC was not found in the sequence
number of real results: 22           ;  ERROR COUNT:  2;  SKIPPED READS:  3;  Execution time: 0.06654691696166992
All  reads:
seq_1, seq_2, seq_3_bad_miss_f, seq_4, seq_5, seq_6_bad_miss_r, seq_7, seq_8, seq_9_bad_rev_prim_first, seq_10, seq_11, seq_12, seq_13, seq_14, seq_15, seq_16, seq_17, seq_18, seq_19, seq_20, seq_21, seq_22, seq_23, seq_24, seq_25
Good reads:
seq_1, seq_2, seq_4, seq_5, seq_7, seq_8, seq_10, seq_11, seq_12, seq_13, seq_14, seq_15, seq_16, seq_17, seq_18, seq_19, seq_20, seq_21, seq_22, seq_23, seq_24, seq_25
Bad reads:
seq_9_bad_rev_prim_first, seq_3_bad_miss_f, seq_6_bad_miss_r


[SeqRecord(seq=Seq('TCCGTAGGTGAACCTGCGGAAAAAAAAAAAAAAAAAAAAAAAAAGCATCGATGA...AGC'), id='seq_1', name='seq_1', description='seq_1 forward=TCCGTAGGTGAACCTGCGG reverse=GCTGCGTTCTTCATCGATGC', dbxrefs=[]),
 SeqRecord(seq=Seq('TCCGTAGGTGAACCTGCGGAAAAAAAAAAAAAAAAAAAAAAAAAGCATCGATGA...AGC'), id='seq_2', name='seq_2', description='seq_2 forward=TCCGTAGGTGAACCTGCGG reverse=GCTGCGTTCTTCATCGATGC', dbxrefs=[]),
 SeqRecord(seq=Seq('TCCGTAGGTGAACCTGCGGAAAAAAAAAAAAAAAAAAAAAAAAAGCATCGATGA...AGC'), id='seq_4', name='seq_4', description='seq_4 forward=TCCGTAGGTGAACCTGCGG reverse=GCTGCGTTCTTCATCGATGC', dbxrefs=[]),
 SeqRecord(seq=Seq('TCCGTAGGTGAACCTGCGGAAAAAAAAAAAAAAAAAAAAAAAAAGCATCGATGA...AGC'), id='seq_5', name='seq_5', description='seq_5 forward=TCCGTAGGTGAACCTGCGG reverse=GCTGCGTTCTTCATCGATGC', dbxrefs=[]),
 SeqRecord(seq=Seq('TCCGTAGGTGAACCTGCGGAAAAAAAAAAAAAAAAAAAAAAAAAGCATCGATGA...AGC'), id='seq_7', name='seq_7', description='seq_7 forward=TCCGTAGGTGAACCTGCGG reverse=GCTGCGTTCTTCATCGATGC', dbxrefs=

## Extract amplicons with hyperex repaired

In [9]:
from Bio import SeqIO
import sys

lenky = {}
seqs_list = []
for n,row in primers_df_filtered.iterrows():
    _,primer_seq,region,_,better_name, = list(row)
    forw = primer_seq.split("/")[0].strip()
    rev = primer_seq.split("/")[1].strip()
    print(better_name)
    res_seq = hyperex_repaired(fasta_seq,forw,rev)
    output_file = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{better_name}.fasta"
    output_dir = f"../datasets/silva_data/amplicons_from_primers_hyperex/{better_name}.fasta"
    with open(output_file, "w") as handle:
        SeqIO.write(res_seq, handle, "fasta-2line")
    print(n)
    lenky[better_name] = len(res_seq)
    seqs_list.append(res_seq)


18S-F___5.8S-R
WARNINGS:726
100600-times  Sequence length is less than 1500 bp. We may not be able to find some regions
30955-times  Region  not found because primers GTAAAAGTCGTAACAAGGTTTC, GTTCAAAGAYTCGATGATTCAC was not found in the sequence
69585-times  Region  not found because primer GTAAAAGTCGTAACAAGGTTTC was not found in the sequence
46-times  Region  not found because primer GTTCAAAGAYTCGATGATTCAC was not found in the sequence
number of real results: 139           ;  ERROR COUNT:  2;  SKIPPED READS:  100587;  Execution time: 10.63282036781311
0
5.8S_Fun___ITS4_Fun
WARNINGS:
100600-times  Sequence length is less than 1500 bp. We may not be able to find some regions
84325-times  Region  not found because primer AGCCTCCGCTTATTGATATGCTTAART was not found in the sequence
16401-times  Region  not found because primers AACTTTYRRCAAYGGATCWCT, AGCCTCCGCTTATTGATATGCTTAART was not found in the sequence
number of real results: 0           ;  ERROR COUNT:  1;  SKIPPED READS:  100726;  Execu

In [ ]:
ids = {}
for p,seq in zip(primers_df_filtered["better_names"],seqs_list):
    ids[p] = [i.id for i in seq]

NameError: name 'seqs_list' is not defined

In [12]:
from itertools import chain
ids_one_list = chain.from_iterable([v for k, v in ids.items()])
from collections import Counter
counts = Counter(ids_one_list)
len(counts),len(fasta_seq)

NameError: name 'ids' is not defined

### use Simera on fastas from hyperex

In [24]:
skipped = []

In [31]:
import os
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from time import sleep
from subprocess import run
import time
from subprocess import TimeoutExpired

# Function to append "_1" to each read name in the FASTA file
def append_suffix_to_fasta(input_file, output_file, suffix="_1"):
    records = list(SeqIO.parse(input_file, "fasta"))
    modified_records = []
    for record in records:
        record.id = f"{record.id}{suffix}"
        record.description = ""  # Clear description to avoid duplication
        modified_records.append(record)
    with open(output_file, "w") as handle:
        SeqIO.write(modified_records, handle, "fasta-2line")

# Get the next available directory number for a primer
def get_next_dir_number(base_dir):
    if not os.path.exists(base_dir):
        return 1
    
    existing_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and d.isdigit()]
    if not existing_dirs:
        return 1
    
    dir_numbers = [int(d) for d in existing_dirs]
    return max(dir_numbers) + 1

# Iterate over each primer and its corresponding file
for numero, row in enumerate(primers_df_filtered.iterrows()):
    if numero < 0: continue
    row = row[1]
    primer_name = row["better_names"]
    primer_seq = row["primers sequences"]
    forward_primer = primer_seq.split("/")[0].strip()
    reverse_primer = primer_seq.split("/")[1].strip()
        # Check if the FASTA file is empty

    #if primer_name != "gITS7___ITS4": print("skipping....") ; continue
    print(primer_name)

    input_file = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{primer_name}.fasta"
    modified_file = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{primer_name}_abund.fasta"
    
    # Base directory for this primer
    primer_base_dir = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_simera/{primer_name}/"
    
    # Get the next sequential directory number
    next_dir_num = get_next_dir_number(primer_base_dir)
    output_dir = f"{primer_base_dir}{next_dir_num}/"
    
    records = list(SeqIO.parse(input_file, "fasta"))

    if len(records) < 10:  # If no records are found, continue to the next iteration
        print(f"Skipping {primer_name} as the file is empty.")
        continue
        
    # Append "_1" to each read name
    append_suffix_to_fasta(input_file, modified_file)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print("simera!!!", primer_name, forward_primer, reverse_primer)
    
    # Run Simera
    command = [
        "../Simera/Simera",
        "-i", modified_file,
        "-o", output_dir,
        "-n", "25",
        "-s", "10000",
        "-l", "0.00005",
        "-c", "10000", # too large values for big fastas (6+Mb) will freeze Simera sometimes ??? dunno why, it fries my pc tho
        "-f", forward_primer, # the limit is 20000 sequences in one fasta file                      
        "-r", reverse_primer,
        "-x"
    ] # Simera is stuck when there is too little seqs in the fasta
    
    try:
        result = run(command, capture_output=False, text=False, timeout=30)  # Timeout after 240 seconds
    except TimeoutExpired:
        print(f"Simera timed out for {primer_name}. Skipping...")
        skipped.append((numero, primer_name))
        continue  # Skip to the next iteration
 
    # Print the result for debugging
    print(f"Processed {primer_name}:")
    print(numero+1, "/", len(primers_df_filtered))
    
    

18S-F___5.8S-R
simera!!! 18S-F___5.8S-R GTAAAAGTCGTAACAAGGTTTC GTTCAAAGAYTCGATGATTCAC

  Simera - simulates chimera formation
  Use option -h for help

Generated 10000 potential chimeras...
PCR Round 25...
Done.
Processed 18S-F___5.8S-R:
1 / 38
5.8S_Fun___ITS4_Fun
Skipping 5.8S_Fun___ITS4_Fun as the file is empty.
58A2F___ITS4
simera!!! 58A2F___ITS4 ATCGATGAAGAACGCAG TCCTCCGCTTATTGATATGC

  Simera - simulates chimera formation
  Use option -h for help

Generated 10000 potential chimeras...
PCR Round 25...
Done.
Processed 58A2F___ITS4:
3 / 38
fITS7___ITS4
simera!!! fITS7___ITS4 GTGARTCATCGAATCTTTG TCCTCCGCTTATTGATATGC

  Simera - simulates chimera formation
  Use option -h for help

Generated 10000 potential chimeras...
PCR Round 25...
Done.
Processed fITS7___ITS4:
4 / 38
fITS9___ITS4
Skipping fITS9___ITS4 as the file is empty.
FSeq___RSeq
Skipping FSeq___RSeq as the file is empty.
gITS7___ITS4
simera!!! gITS7___ITS4 GTGARTCATCGARTCTTTG TCCTCCGCTTATTGATATGC

  Simera - simulates chimera

In [29]:
set(skipped)

{(8, 'gITS7ngs___ITS4ngsUni'),
 (9, 'ITS1___ITS2'),
 (10, 'ITS1___ITS4'),
 (21, 'ITS1FI2___5.8S'),
 (22, 'ITS1FI2___ITS2'),
 (24, 'ITS1ngs___ITS2'),
 (33, 'ITS7o___ITS4')}

## Cut primers with cutadapt - cut the Simera output - I should do it at the end

In [33]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
from subprocess import run

# Function to sanitize sequence IDs and descriptions
def sanitize_fasta_records(records):
    sanitized_records = []
    for record in records:
        sanitized_id = ''.join(c if c.isascii() or c in "-_." else "_" for c in record.id)
        sanitized_description = ''.join(c if c.isascii() or c in "-_." else "_" for c in record.description)
        sanitized_records.append(SeqRecord(Seq(str(record.seq)), id=sanitized_id, description=sanitized_description))
    return sanitized_records

# Iterate over each primer and its corresponding file
for _, row in primers_df_filtered.iterrows():
    primer_name = row["better_names"]
    primer_seq = row["primers sequences"]
    #if primer_name != "gITS7___ITS4": continue #########!!!!!!!!!!!

    simera_dir = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_simera/{primer_name}/"
    try:
        for time_dir in os.listdir(simera_dir):
            time_dir = simera_dir + time_dir
            
            input_file = time_dir + "/samp_all_seqs.fa"
            sanitized_file = time_dir + "/samp_all_seqs_sanitized.fa"
            output_file = time_dir + "/samp_all_seqs_cutadapt.fa"
            
            if not os.path.exists(input_file):
                print(f"Input file {input_file} does not exist. Skipping...")
                continue
            # Sanitize the input FASTA file
            records = list(SeqIO.parse(input_file, "fasta"))
            sanitized_records = sanitize_fasta_records(records)
            with open(sanitized_file, "w") as handle:
                SeqIO.write(sanitized_records, handle, "fasta")
            
            # Run cutadapt to cut primers
            command = [
                "cutadapt",
                "-g", primer_seq.split("/")[0].strip(),  # Forward primer
                "-a", primer_seq.split("/")[1].strip(),  # Reverse primer
                "-o", output_file,
                sanitized_file
            ]
            result = run(command, capture_output=True, text=True)
            
            # Print the result for debugging
            print(f"Processed {primer_name}:")
            print(result.stdout)
            print(result.stderr)
    except: print(primer_name, "error - not found")

Processed 18S-F___5.8S-R:
This is cutadapt 5.0 with Python 3.12.2
Command line parameters: -g GTAAAAGTCGTAACAAGGTTTC -a GTTCAAAGAYTCGATGATTCAC -o ../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_simera/18S-F___5.8S-R/1/samp_all_seqs_cutadapt.fa ../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_simera/18S-F___5.8S-R/1/samp_all_seqs_sanitized.fa
Processing single-end reads on 1 core ...

=== Summary ===

Total reads processed:                     369
Reads with adapters:                       369 (100.0%)
Reads written (passing filters):           369 (100.0%)

Total basepairs processed:       133,210 bp
Total written (filtered):        125,092 bp (93.9%)

=== Adapter 1 ===

Sequence: GTAAAAGTCGTAACAAGGTTTC; Type: regular 5'; Length: 22; Trimmed: 369 times

Minimum overlap: 3
No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-22 bp: 2

Overview of removed sequences
length	c

### RUN vsearch uchime on Simera data

In [11]:
#simera_fasta_path = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_simera/"
#simera_fasta_file = f"gITS7___ITS4/20250325_182506/all_seqs_cutadapt.fa"
simera_fasta_path = ""
simera_fasta_file = f"../my_data/all_simera_for_uchime.fasta"
from pprint import pprint
simera_seqs = [(i.id,i.seq) for i in SeqIO.parse(simera_fasta_path+simera_fasta_file, "fasta")]
print(len(simera_seqs),"len")
pprint(simera_seqs[:20])
#simera_seqs = list(filter(lambda x: int(x[0].split("_")[-1]) > 0, simera_seqs))
simera_seqs = list(filter(lambda x: int(x[0].split("_-_")[0].split("_")[-1]) > 0, simera_seqs))

print(len(simera_seqs),"len after filtering")


simera_seqs_for_uchime=list(map(lambda x: ("".join(x[0].split("_-_")[0])+";size="+x[0].split("_-_")[0].split("_")[-1]+";"+x[0].split("_-_")[1],x[1]),simera_seqs))

print(simera_seqs_for_uchime[:2])
from Bio.SeqRecord import SeqRecord

output_file = simera_fasta_path + ".".join(simera_fasta_file.split(".")[:-1]) + "_for_uchime.fa"
seq_records = []
for header, seq in simera_seqs_for_uchime:
    record = SeqRecord(seq, id=header, description="")
    seq_records.append(record)
with open(output_file, 'w') as output_handle:
    SeqIO.write(seq_records, output_handle, 'fasta-2line')
print(f"output_file: {output_file}")

35456 len
[('chimeraREV_253_1_-_18S-F___5.8S-R',
  Seq('CGTAGGTGAACCTGCGGAAGGATCATTATTGAATAAACCTGATGTGGTTGTTGC...AAC')),
 ('chimeraREV_287_1_-_18S-F___5.8S-R',
  Seq('CGTAGGTGAACCTGCGGAAGGATCATTATCGAAATCGAAGAGGTGGAGGGACTG...GAA')),
 ('chimeraFWD_150_5_-_18S-F___5.8S-R',
  Seq('CGTAGGTGAACCTGCGGAAGGATCATTATTGAATAAACCTGATGTGGTTGTAGC...AAC')),
 ('chimeraFWD_364_1_-_18S-F___5.8S-R',
  Seq('CGTAGGTGAACCTGCGGAAGGATCATTATTGAATAAACCTGGTGGGTTGTAGCT...AAC')),
 ('chimeraREV_164_2_-_18S-F___5.8S-R',
  Seq('CGTAGGTGAACCTGCGGAAGGATCATTACCAATAAGGCTACCTTGGACCAAAAG...AAC')),
 ('Agaricaceae_sp|OM972416|SH1182565.09FU|reps|k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Agaricaceae;g__unidentified;s__Agaricaceae_sp_1_12_4_-_18S-F___5.8S-R',
  Seq('CGTAGGTGAACCTGCGGAAGGATCATTATTGAATAAACCTGGTGGGTTGTCGCT...CAA')),
 ('chimeraREV_147_2_-_18S-F___5.8S-R',
  Seq('CGTAGGTGAACCTGCGGAAGGATCATTATCGATTTCAAAGGATGTGCTGGCTGC...AAC')),
 ('chimeraREV_372_1_-_18S-F___5.8S-R',
  Seq('CGTAGGTGAACCTGCGGAAGGATCAT

In [56]:
output_file

'../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_simera/gITS7___ITS4/20250325_182506/all_seqs_cutadapt_for_uchime.fa'

In [62]:
import subprocess
out_chim_detect_dir = "/".join(output_file.split("/")[:-1]) + "/uchime_out/"
# Define the command and its arguments
command = [
    "/home/mg/vsearch-2.29.2-linux-x86_64/bin/vsearch",
    "--uchime_denovo", output_file,
    "--chimeras", out_chim_detect_dir+"chimeras_uch_de.fa",
    "--nonchimeras", out_chim_detect_dir+"nonchimeras_uch_de.fa",
    "--fasta_score"
]

# Run the command
try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print("Command executed successfully.")
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
except subprocess.CalledProcessError as e:
    print("Error occurred while executing the command.")
    print("Return code:", e.returncode)
    print("STDOUT:", e.stdout)
    print("STDERR:", e.stderr)

Command executed successfully.
STDOUT: 
STDERR: vsearch v2.29.2_linux_x86_64, 15.4GB RAM, 16 cores
https://github.com/torognes/vsearch

Reading file ../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_simera/gITS7___ITS4/20250325_182506/all_seqs_cutadapt_for_uchime.fa 100%
9948345 nt in 33066 seqs, min 1, max 927, avg 301
Masking 100%
Sorting by abundance 100%
Counting k-mers 100%
Detecting chimeras 100%
Found 10259 (31.0%) chimeras, 21703 (65.6%) non-chimeras,
and 1104 (3.3%) borderline sequences in 33066 unique sequences.
Taking abundance information into account, this corresponds to
2093128 (1.2%) chimeras, 173234305 (98.6%) non-chimeras,
and 327858 (0.2%) borderline sequences in 175655291 total sequences.



In [7]:
out_chim_detect_dir = "../my_data/all_simera_for_uchime_for_uchime_after_uchime/"
chimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"chimeras_chi_de.fa", "fasta"))
nonchimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"nonchimeras_chi_de.fa", "fasta"))
chimeras_uch_de_good = [i for i in chimeras_uch_de if "chimera" in i.description]
nonchimeras_uch_de_good = [i for i in nonchimeras_uch_de if "chimera" not in i.description]
#pprint(chimeras_uch_de_good[:2]);pprint(nonchimeras_uch_de_good[:2])
acc = (len(nonchimeras_uch_de_good)+len(chimeras_uch_de_good))/(len(nonchimeras_uch_de)+len(chimeras_uch_de))
print("chim_denovo",acc)

out_chim_detect_dir = "../my_data/all_simera_for_uchime_for_uchime_after_uchime/"
chimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"chimeras_uch_de.fa", "fasta"))
nonchimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"nonchimeras_uch_de.fa", "fasta"))
chimeras_uch_de_good = [i for i in chimeras_uch_de if "chimera" in i.description]
nonchimeras_uch_de_good = [i for i in nonchimeras_uch_de if "chimera" not in i.description]
#pprint(chimeras_uch_de_good[:2]);pprint(nonchimeras_uch_de_good[:2])
acc = (len(nonchimeras_uch_de_good)+len(chimeras_uch_de_good))/(len(nonchimeras_uch_de)+len(chimeras_uch_de))
print("uchime_denovo",acc)

out_chim_detect_dir = "../my_data/all_simera_for_uchime_for_uchime_after_uchime/"
chimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"chimeras_uch_ref_p_clust.fa", "fasta"))
nonchimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"nonchimeras_uch_ref_p_clust.fa", "fasta"))
chimeras_uch_de_good = [i for i in chimeras_uch_de if "chimera" in i.description]
nonchimeras_uch_de_good = [i for i in nonchimeras_uch_de if "chimera" not in i.description]
#pprint(chimeras_uch_de_good[:2]);pprint(nonchimeras_uch_de_good[:2])
acc = (len(nonchimeras_uch_de_good)+len(chimeras_uch_de_good))/(len(nonchimeras_uch_de)+len(chimeras_uch_de))
print("uchime_ref",acc)

out_chim_detect_dir = "../my_data/all_simera_for_uchime_for_uchime_after_uchime/"
chimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"chimeras_uch_ref_unite_all.fa", "fasta"))
nonchimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"nonchimeras_uch_ref_unite_all.fa", "fasta"))
chimeras_uch_de_good = [i for i in chimeras_uch_de if "chimera" in i.description]
nonchimeras_uch_de_good = [i for i in nonchimeras_uch_de if "chimera" not in i.description]
#pprint(chimeras_uch_de_good[:2]);pprint(nonchimeras_uch_de_good[:2])
acc = (len(nonchimeras_uch_de_good)+len(chimeras_uch_de_good))/(len(nonchimeras_uch_de)+len(chimeras_uch_de))
print("uchime_ref all unite",acc)

out_chim_detect_dir = "../my_data/all_simera_for_uchime_for_uchime_after_uchime/"
chimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"chimeras_uch_ref_unite_its2.fa", "fasta"))
nonchimeras_uch_de = list(SeqIO.parse(out_chim_detect_dir+"nonchimeras_uch_ref_unite_its2.fa", "fasta"))
chimeras_uch_de_good = [i for i in chimeras_uch_de if "chimera" in i.description]
nonchimeras_uch_de_good = [i for i in nonchimeras_uch_de if "chimera" not in i.description]
#pprint(chimeras_uch_de_good[:2]);pprint(nonchimeras_uch_de_good[:2])
acc = (len(nonchimeras_uch_de_good)+len(chimeras_uch_de_good))/(len(nonchimeras_uch_de)+len(chimeras_uch_de))
print("uchime_ref its2 unite",acc)

chim_denovo 0.5780967960288809
uchime_denovo 0.5349314265619506
uchime_ref 0.4687110304954087
uchime_ref all unite 0.534196099577604
uchime_ref its2 unite 0.4527647126304921


In [60]:
import os
import sys
import shutil

# Display the current PATH environment variable
print("Current PATH:")
path_entries = os.environ.get('PATH', '').split(':')
for i, path in enumerate(path_entries):
    print(f"{i+1}: {path}")


Current PATH:
1: /home/mg/miniconda3/envs/chim/bin
2: /home/mg/.vscode-server/bin/ddc367ed5c8936efe395cffeec279b04ffd7db78/bin/remote-cli
3: /home/mg/.cargo/bin
4: /home/mg/miniconda3/envs/chim/bin
5: /home/mg/miniconda3/condabin
6: /usr/local/sbin
7: /usr/local/bin
8: /usr/sbin
9: /usr/bin
10: /sbin
11: /bin
12: /usr/games
13: /usr/local/games
14: /mnt/c/Program Files/Common Files/Oracle/Java/javapath
15: /mnt/c/Program Files (x86)/Common Files/Oracle/Java/javapath
16: /mnt/c/Windows/system32
17: /mnt/c/Windows
18: /mnt/c/Windows/System32/Wbem
19: /mnt/c/Windows/System32/WindowsPowerShell/v1.0
20: /mnt/c/Windows/System32/OpenSSH
21: /mnt/c/Program Files/dotnet
22: /mnt/c/Program Files/Mullvad VPN/resources
23: /mnt/c/Program Files/MIT/Kerberos/bin
24: /mnt/c/Users/hubmich/scoop/shims
25: /mnt/c/Users/hubmich/AppData/Local/Programs/Python/Launcher
26: /mnt/c/Users/hubmich/AppData/Local/Microsoft/WindowsApps
27: /mnt/c/Users/hubmich/AppData/Local/Programs/Ollama
28: /mnt/c/Users/hubmich

### END

## OLD versions:

In [ ]:
# old version
import os
import subprocess
import re
from collections import Counter
import sys

results = []  # List to store the content of hyperex_out.fa files
warnings = []  # List to store the warnings
errors = []  # List to store the errors
hyperex_out_fa = f"hyperex_temp_out.fa"
hyperex_log = f"hyperex.log"
hyperex_gff = f"hyperex_temp_out.gff"

# Clean up any existing temporary files
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)

# Command template for hyperex
command = ["hyperex", "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC", "-p", "hyperex_temp_out"]

lenka = len(fasta_seq)
for i, record in enumerate(fasta_seq):
    if i % 50 == 0: print(f"{i}/{lenka}", end="\r")
    if i >0: command = ["hyperex", "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC", "-p", "hyperex_temp_out","--force"] 
    # Convert the record to FASTA format string
    fasta_data = f">{record.id}\n{str(record.seq)}\n"
    
    # Run hyperex with the FASTA data passed via stdin
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=fasta_data)
    
    # Collect warnings and errors
    warnings.extend(list(map(lambda x: x.replace("\x1b[0m]", ""), re.findall("WARN(.+)", stdout))))
    errors.extend(stderr)
    
    # Read results if the output file exists
    if os.path.exists(hyperex_out_fa):
        with open(hyperex_out_fa, "r") as fa_file:
            results.append(fa_file.read())

# Clean up any remaining temporary files
if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)

print()
real_results = [i for i in results if len(i) > 0]
print("number of real results:", len(real_results))
warning_counts = Counter(warnings)
error_counts = Counter(errors)
print("ERRORS:", error_counts, "WARNINGS:", warning_counts, sep="\n")


In [ ]:
# old version 2
import os
import subprocess
#from time import sleep
#import sys
import re
from collections import Counter
results = []  # List to store the content of hyperex_out.fa files
warnings = []  # List to store the warnings
errors = []  # List to store the errors
hyperex_out_fa = f"hyperex_temp_out.fa"
hyperex_log = f"hyperex.log"
hyperex_gff = f"hyperex_temp_out.gff"
temp_filename = f"hyperex_temp_res.fasta"
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(temp_filename): os.remove(temp_filename)  # Delete the temporary input file

with open(temp_filename, "w") as temp_fasta:
    SeqIO.write(fasta_seq[0], temp_fasta, "fasta")
command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
warnings.extend(list(map(lambda x: x.replace("\x1b[0m]",""), re.findall("WARN(.+)",result.stdout))))
errors.extend(result.stderr)
if os.path.exists(hyperex_out_fa):
    with open(hyperex_out_fa, "r") as fa_file:
        results.append(fa_file.read())
#print(result.stdout)
#print(result.stderr)
lenka = len(fasta_seq)
command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename,"--force"]
for i, record in enumerate(fasta_seq[1:]):
    #temp_filename = f"hyperex_temp_res.fasta"
    if i % 50 == 0: print(f"{i}/{lenka}",end="\r")
    with open(temp_filename, "w") as temp_fasta:
        SeqIO.write(record, temp_fasta, "fasta")
    #if i == 0:command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    warnings.extend(list(map(lambda x: x.replace("\x1b[0m]",""), re.findall("WARN(.+)",result.stdout))))
    errors.extend(result.stderr)
    #if os.path.exists(hyperex_out_fa):
    with open(hyperex_out_fa, "r") as fa_file:
        results.append(fa_file.read())

if os.path.exists(hyperex_out_fa):os.remove(hyperex_out_fa)  
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(temp_filename): os.remove(temp_filename)  # Delete the temporary input file

print()
real_results = [i for i in results if len(i) >0]
print("number of real results:",len(real_results))
warning_counts = Counter(warnings)
error_counts = Counter(errors)
print("ERRORS:",list(error_counts),"WARNINGS:","\n".join(list(warning_counts)),sep="\n")



In [ ]:
from math import sqrt
x = 5
a = lambda x: 0
b = lambda x: 1
c = lambda x: x
d = lambda x: sqrt(x)
e = lambda x: x**(1/3)
f = lambda x: sqrt(sqrt(x))
g = lambda x: x**1.1
h = lambda x: x**1.2
i = lambda x: x**1.3
j = lambda x: x**1.5
k = lambda x: x**2


print(a(x))
print(b(x))
print(c(x))
print(d(x))
print(e(x))
print(f(x))


0
1
5
2.23606797749979
1.7099759466766968
1.4953487812212205


In [10]:
a ="""724243116		602368116
605418059		603515119
725531547		607954746
724 666 277		602533799
737363099		736427302
777049546		776338316
728604166		
732876652		728839198
776333462		777333264
776036182		606061648
724 666 277		608 311 383
725531547		607954746
731 624 508		""".replace(" ","").split("\n")
b= list(filter(lambda x: len(x)>0,"\t".join(a).split("\t")))
c=set(b)
",".join(c)
len(c)

21